The baseline Logistic Regression algorithm we'll try first
from https://realpython.com/python-keras-text-classification/

In [40]:
import sqlite3
import pandas as pd

df = pd.read_csv('distinct_data.csv', index_col=0)

In [41]:
df.head(n=2)

,function,isVulnerable,tokenized_functions
0,static void ipcomp_free_scratches(void)\n{\n\t...,0,"['static', 'void', 'ipcomp_free_scratches', '(..."
1,static void ipcomp_free_scratches(void)\n{\n\t...,1,"['static', 'void', 'ipcomp_free_scratches', '(..."


In [42]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy(doc):
    return doc

vectorizer = CountVectorizer(min_df=0, tokenizer=None, preprocessor=None)
vectorizer.fit(df['tokenized_functions'])


CountVectorizer(min_df=0)

In [43]:
vectorizer.vocabulary_

{'static': 52395,
 'void': 59538,
 'ipcomp_free_scratches': 26729,
 'int': 25786,
 '__percpu': 1436,
 'scratches': 48131,
 'if': 24334,
 'ipcomp_scratch_users': 26730,
 'return': 45471,
 'ipcomp_scratches': 26731,
 'for_each_possible_cpu': 18866,
 'vfree': 58844,
 'per_cpu_ptr': 40257,
 'free_percpu': 19178,
 'null': 37162,
 '__exit': 1176,
 'nbd_cleanup': 34720,
 'struct': 52721,
 'nbd_device': 34733,
 'nbd': 34706,
 'list_head': 29662,
 'del_list': 11920,
 'genl_unregister_family': 19976,
 'nbd_genl_family': 34740,
 'nbd_dbg_close': 34726,
 'mutex_lock': 34348,
 'nbd_index_mutex': 34743,
 'idr_for_each': 24030,
 'nbd_index_idr': 34742,
 'nbd_exit_cb': 34736,
 'mutex_unlock': 34354,
 'while': 60232,
 'list_empty': 29646,
 'list_first_entry': 29649,
 'list': 29637,
 'list_del_init': 29643,
 'refcount_read': 44587,
 'refs': 44600,
 'printk': 41991,
 'kern_err': 28235,
 'possibly': 41602,
 'leaking': 29388,
 'device': 12360,
 'nbd_put': 34747,
 'destroy_workqueue': 12108,
 'nbd_del_wq': 

In [44]:
vectorizer.transform(df['tokenized_functions']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [45]:
from sklearn.model_selection import train_test_split

functions = df['tokenized_functions'].values
y = df['isVulnerable'].values

functions_train, functions_test, y_train, y_test = train_test_split(functions, y, test_size=0.2, random_state=42)

In [46]:
vectorizer = CountVectorizer(min_df=0, tokenizer=None, preprocessor=None)
vectorizer.fit(functions_train)

X_train = vectorizer.transform(functions_train)
X_test = vectorizer.transform(functions_test)

In [47]:
X_train

<7281x60179 sparse matrix of type '<class 'numpy.int64'>'
	with 423690 stored elements in Compressed Sparse Row format>

In [50]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.3371773750686436


In [51]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]
print(X_train.shape[1])

2023-04-29 11:34:48.477390: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 11:34:48.509944: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 11:34:48.510153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 11:34:49.158481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


60179


In [52]:
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


2023-04-29 11:35:50.841886: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-29 11:35:50.844356: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [53]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                601800    
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 601,811
Trainable params: 601,811
Non-trainable params: 0
_________________________________________________________________


In [54]:
history = model.fit(X_train, y_train, epochs=100, verbose=False, validation_data=(X_test, y_test), batch_size=10)
from keras.backend import clear_session
clear_session()

loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

2023-04-29 11:42:31.990119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [7281]
	 [[{{node Placeholder/_1}}]]
2023-04-29 11:42:31.990330: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [7281]
	 [[{{node Placeholder/_1}}]]
2023-04-29 11:42:37.442307: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1821]
	

Training Accuracy: 0.6550
Testing Accuracy:  0.3954


2023-04-29 11:50:50.033898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1821]
	 [[{{node Placeholder/_1}}]]
